In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import py_fort_myrmidon as fm
import datetime
import networkx as nx
import networkx.algorithms.community as nxc
import statistics
import scipy.stats as stats
import os
from datetime import datetime, date, timedelta
import pickle
import warnings
import json
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Set working directory and open list of myrmidon files

working_dir = '/media/ll16598/EG_DATA-7/NTM/'
myrm_list = sorted([ s for s in os.listdir(working_dir) if s[15:17] == '_z'])

In [3]:
myrm_list = [m for m in myrm_list if '_nn' in m]


## Test mode (optional)

To reduce the computational cost of the script, turn on the test mode option. This will run the analysis only on the first 2 myrmidon files. 

In [4]:
test_mode = False
#test_mode = True

In [5]:
# class Interaction:
#     def __init__(self, duration, zone, start, end):
#         self.duration = duration
#         self.zone = zone
#         self.start = start
#         self.end = end

#     def __repr__(self):
#         return f"Duration: {self.duration}, Zone: {self.zone}, Start: {self.start}, End: {self.end}"


class Interaction:
    def __init__(self, duration, zone, start, end):
        self.duration = duration
        self.zone = zone
        self.start = start
        self.end = end

    def to_dict(self):
        return {
            'duration': self.duration,
            'zone': self.zone,
            'start': self.start,
            'end': self.end
        }

    def __repr__(self):
        return f"Duration: {self.duration}, Zone: {self.zone}, Start: {self.start}, End: {self.end}"

    
def temporal_network(exp, start, end,min_cum_duration, frm_rate, link_type, nest_focus, save=False):
    ant_id1=[]
    ant_id2=[]
    durations=[]
    zones=[]
    time_start=[]
    time_end=[]
    interaction_list=[]
    interactions=fm.Query.ComputeAntInteractions(exp, start=start, end=end, maximumGap=fm.Duration(max_gap*10**9),
                                             reportFullTrajectories= False)[1]
    
    TimeToFrame = {fm.Time.ToTimestamp(frm[0].FrameTime): i + 1 
               for i, frm in enumerate(fm.Query.CollideFrames(exp,start=start,end=end))}
    for i in interactions:
        zone1=tuple(i.Trajectories[1].Zones)[0]
        try:
            zone2=tuple(i.Trajectories[1].Zones)[1]
            zone=zone2
        except IndexError:
            zone=zone1
        
        #zone 2 is the arena
        if nest_focus and zone1==2:
            continue
        if link_type == 'length_inter':
        # WEIGHTS: cumulative interaction time (s)
            weight=(TimeToFrame[fm.Time.ToTimestamp(i.End)] - \
                                                TimeToFrame[fm.Time.ToTimestamp(i.Start)]) * 1\
            / frm_rate
        elif link_type == '#inter':
            weight=1
        ant_id1.append(i.IDs[0])
        ant_id2.append(i.IDs[1])
        durations.append(weight)
        zones.append(zone)
        time_start.append(TimeToFrame[fm.Time.ToTimestamp(i.Start)] )
        time_end.append(TimeToFrame[fm.Time.ToTimestamp(i.End)] )
        interaction_list.append(tuple([i.IDs[0], i.IDs[1], weight, zone]))

    # transform into temporal network
    G = nx.MultiGraph()

    # Add nodes and edges
    for i in range(len(ant_id1)):
        G.add_node(ant_id1[i])
        G.add_node(ant_id2[i])
        interaction = Interaction(durations[i], zones[i], time_start[i], time_end[i])
        G.add_edge(ant_id1[i], ant_id2[i], interaction=json.dumps(interaction.to_dict()))
    for node in G.nodes():
        G.nodes[node]['pathogen_load'] = 0
        
    df = pd.DataFrame({
    'ant_id1': ant_id1,
    'ant_id2': ant_id2,
    'durations': durations,
    'zones': zones,
    'time_start': time_start,
    'time_end': time_end
    })
    return G, df

In [8]:
## ========= PARAMETERS========= 
# Frame rate
save_df=False
frm_rate = 6
#how long to make the temporal network over
hours_g=24
# max gap for different interactions (s)
max_gap = 10

# minimum interaction weight (OPTIONAL, set=0 if not wanted)
min_cum_duration = 0 

# Read file with modes of communities number computed    
#mode_communities_dic = pd.read_pickle(r'data/mode_communities.pkl')

# ========= Edges weights type ==========
# Decide what type of weight for the edges of the social network 
#     'length_inter': cumulative interaction time (s))    
#     '#inter': number of interactions    
#use length_inter for transmission
link_type = '#inter'#'length_inter'

# ========= Nest focus zone ==========
# Set weather to only consider interactions happening within the nest zone
#     'True': Nest only
#     'False': All interactions considered
nest_focus = True

#data_saving_name = 'data/polish_data_test' + link_type + '_NF_F.pkl'

if test_mode:
    #myrm_list = [m for m in myrm_list if 'test' in m]
    myrm_list = myrm_list[20:22]

# Loop through the myrmidon files
for myr in range(0, len(myrm_list)):
    myrm_file=myrm_list[myr]
    if int(myrm_file[8:10])==41 or int(myrm_file[8:10])<13: 
        continue
    print(myrm_file)
    exp = fm.Experiment.Open(working_dir + myrm_file)

    start_date = (fm.Time.ToDateTime(fm.Query.GetDataInformations(exp).End) +
                 timedelta(days = -1)).strftime("%Y-%m-%d")

    # ======== Total cumulative time window ========
    # Define the start/end of the entire time window to be considered to the analysis
    # ie from 9:00am of start_date to 9:00am of the following day
    #for the temporal network we use the entire time window
    start = fm.Time(datetime.fromisoformat(start_date + 'T09:00:00'))  
    end = start.Add(fm.Duration(60 * 60 * hours_g * 10**9))
    G, df = temporal_network(exp, start, end, min_cum_duration, frm_rate, link_type, nest_focus=nest_focus, save=False)
    if link_type == 'length_inter':
        nx.write_graphml(G, working_dir+'duration_temporal_networks/'+myrm_file+ "_graph.xml")
    elif link_type == '#length':
        nx.write_graphml(G, working_dir+'number_temporal_networks/'+myrm_file+ "_graph.xml")
    if save_df:
        df.to_csv(working_dir+'edge_lists/'+myrm_file+ "_interactions.csv")


EG_NTM_s13_DEHa_zones_luke_2023_nn.myrmidon


Colliding frames: 100%|███████████▉| 1439/1440 [00:35<00:00, 40.88tracked min/s]


EG_NTM_s13_DEHb_zones_luke_2023_nn.myrmidon


Colliding frames: 100%|███████████▉| 1439/1440 [00:37<00:00, 38.76tracked min/s]


EG_NTM_s14_MODa_zones_luke_2023_nn.myrmidon


...

Colliding frames: 100%|███████████▉| 1439/1440 [00:31<00:00, 46.07tracked min/s]


EG_NTM_s56_DENa_zones_luke_2023_nn.myrmidon


Colliding frames: 100%|███████████▉| 1439/1440 [00:32<00:00, 44.96tracked min/s]


EG_NTM_s56_DENb_zones_luke_2023_nn.myrmidon


Colliding frames: 100%|███████████▉| 1439/1440 [00:33<00:00, 42.95tracked min/s]


Very important if I correspond to spatial network that ant_ids-1